In [2]:
!pip install findspark
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="myAppName")
sc

<SparkContext master=local[*] appName=myAppName>

In [17]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, count, expr, avg, lit, max, min

# Crear una instancia de SparkSession
spark = SparkSession.builder.appName("Ejercicio").getOrCreate()

In [29]:
# Definir los datos de personas
data = [("Alice", 25, "New York"),
        ("Bob", 30, "Los Angeles"),
        ("Charlie", 22, "Chicago")]

In [30]:
# 1. Crear un DataFrame con los datos de personas
df = spark.createDataFrame(data, ["Nombre", "Edad", "Ciudad"])

In [31]:
# 2. Mostrar solo los nombres de las personas del DataFrame
df.select("Nombre").show()

+-------+
| Nombre|
+-------+
|  Alice|
|    Bob|
|Charlie|
+-------+



In [32]:
# 3. Filtrar personas cuya edad sea mayor o igual a 25
df.filter(df.Edad >= 25).show()

+------+----+-----------+
|Nombre|Edad|     Ciudad|
+------+----+-----------+
| Alice|  25|   New York|
|   Bob|  30|Los Angeles|
+------+----+-----------+



In [33]:
# 4. Agregar una nueva columna "Pais" con un valor constante para todas las filas
df = df.withColumn("Pais", lit("EEUU"))
df.show()

+-------+----+-----------+----+
| Nombre|Edad|     Ciudad|Pais|
+-------+----+-----------+----+
|  Alice|  25|   New York|EEUU|
|    Bob|  30|Los Angeles|EEUU|
|Charlie|  22|    Chicago|EEUU|
+-------+----+-----------+----+



In [34]:
# 5. Calcular el promedio de edad de todas las personas
df.select(avg("Edad")).show()

+------------------+
|         avg(Edad)|
+------------------+
|25.666666666666668|
+------------------+



In [35]:
# 6. Ordenar el DataFrame por edad en orden descendente
df.orderBy(df.Edad.desc()).show()

+-------+----+-----------+----+
| Nombre|Edad|     Ciudad|Pais|
+-------+----+-----------+----+
|    Bob|  30|Los Angeles|EEUU|
|  Alice|  25|   New York|EEUU|
|Charlie|  22|    Chicago|EEUU|
+-------+----+-----------+----+



In [36]:
# 7. Agrupar por ciudad y calcular la cantidad de personas en cada ciudad
df.groupBy("Ciudad").agg(count("*").alias("Cantidad")).show()

+-----------+--------+
|     Ciudad|Cantidad|
+-----------+--------+
|   New York|       1|
|Los Angeles|       1|
|    Chicago|       1|
+-----------+--------+



In [37]:
# 8. Renombrar la columna "Nombre" a "NombreCompleto"
df = df.withColumnRenamed("Nombre", "NombreCompleto")
df.show()

+--------------+----+-----------+----+
|NombreCompleto|Edad|     Ciudad|Pais|
+--------------+----+-----------+----+
|         Alice|  25|   New York|EEUU|
|           Bob|  30|Los Angeles|EEUU|
|       Charlie|  22|    Chicago|EEUU|
+--------------+----+-----------+----+



In [38]:
# 9. Crear una copia del DataFrame y Eliminar la columna "Edad" de la copia del DataFrame
df_copia = df.drop("Edad")
df_copia.show()

+--------------+-----------+----+
|NombreCompleto|     Ciudad|Pais|
+--------------+-----------+----+
|         Alice|   New York|EEUU|
|           Bob|Los Angeles|EEUU|
|       Charlie|    Chicago|EEUU|
+--------------+-----------+----+



In [39]:
# 10. Realizar una consulta SQL en el DataFrame para seleccionar personas mayores de 20 años
df.createOrReplaceTempView("personas")
resultados_sql = spark.sql("SELECT * FROM personas WHERE Edad >= 20")
resultados_sql.show()

+--------------+----+-----------+----+
|NombreCompleto|Edad|     Ciudad|Pais|
+--------------+----+-----------+----+
|         Alice|  25|   New York|EEUU|
|           Bob|  30|Los Angeles|EEUU|
|       Charlie|  22|    Chicago|EEUU|
+--------------+----+-----------+----+



In [40]:
# 11. Calcular la suma total de todas las edades.
df.selectExpr("sum(Edad)").show()

+---------+
|sum(Edad)|
+---------+
|       77|
+---------+



In [41]:
# 12. Calcular la edad mínima y máxima de todas las personas
df.select(min("Edad"), max("Edad")).show()

+---------+---------+
|min(Edad)|max(Edad)|
+---------+---------+
|       22|       30|
+---------+---------+



In [42]:
# 13. Filtrar personas cuya ciudad de residencia sea "Chicago" y edad sea menor de 30
df.filter((df.Ciudad == "Chicago") & (df.Edad < 30)).show()

+--------------+----+-------+----+
|NombreCompleto|Edad| Ciudad|Pais|
+--------------+----+-------+----+
|       Charlie|  22|Chicago|EEUU|
+--------------+----+-------+----+



In [44]:
# 14. Agregar una nueva columna "EdadDuplicada" que contenga el doble de la edad
df = df.withColumn("EdadDuplicada", col("Edad") * 2)
df.show()

+--------------+----+-----------+----+-------------+
|NombreCompleto|Edad|     Ciudad|Pais|EdadDuplicada|
+--------------+----+-----------+----+-------------+
|         Alice|  25|   New York|EEUU|           50|
|           Bob|  30|Los Angeles|EEUU|           60|
|       Charlie|  22|    Chicago|EEUU|           44|
+--------------+----+-----------+----+-------------+



In [45]:
# 15. Convertir todas las edades en años a meses
df = df.withColumn("EdadEnMeses", col("Edad") * 12)
df.show()

+--------------+----+-----------+----+-------------+-----------+
|NombreCompleto|Edad|     Ciudad|Pais|EdadDuplicada|EdadEnMeses|
+--------------+----+-----------+----+-------------+-----------+
|         Alice|  25|   New York|EEUU|           50|        300|
|           Bob|  30|Los Angeles|EEUU|           60|        360|
|       Charlie|  22|    Chicago|EEUU|           44|        264|
+--------------+----+-----------+----+-------------+-----------+



In [48]:
# 16. Contar el número total de personas en el DataFrame
total_personas = df.count()
print("Número total de personas en el DataFrame:", total_personas)


Número total de personas en el DataFrame: 3


In [49]:
# 17. Filtrar personas cuya edad sea un número par
df.filter(df.Edad % 2 == 0).show()

+--------------+----+-----------+----+-------------+-----------+
|NombreCompleto|Edad|     Ciudad|Pais|EdadDuplicada|EdadEnMeses|
+--------------+----+-----------+----+-------------+-----------+
|           Bob|  30|Los Angeles|EEUU|           60|        360|
|       Charlie|  22|    Chicago|EEUU|           44|        264|
+--------------+----+-----------+----+-------------+-----------+



In [50]:
# 18. Calcular la cantidad de personas por rango de edades (0-20, 21-40, 41-60, 61+)
df.groupBy(expr("CASE WHEN Edad BETWEEN 0 AND 20 THEN '0-20' WHEN Edad BETWEEN 21 AND 40 THEN '21-40' WHEN Edad BETWEEN 41 AND 60 THEN '41-60' ELSE '61+' END AS RangoEdades")).agg(count("*").alias("Cantidad")).show()

+-----------+--------+
|RangoEdades|Cantidad|
+-----------+--------+
|      21-40|       3|
+-----------+--------+



In [51]:
# 19. Contar cuántas personas tienen el mismo nombre
df.groupBy("NombreCompleto").agg(count("*").alias("Cantidad")).show()

+--------------+--------+
|NombreCompleto|Cantidad|
+--------------+--------+
|         Alice|       1|
|           Bob|       1|
|       Charlie|       1|
+--------------+--------+



In [53]:
# 20. Concatenar las columnas "Nombre" y "Ciudad" en una nueva columna llamada "InformacionPersonal"
df = df.withColumn("InformacionPersonal", concat(col("NombreCompleto"), lit(", "), col("Ciudad"))).show()

+--------------+----+-----------+----+-------------+-----------+-------------------+
|NombreCompleto|Edad|     Ciudad|Pais|EdadDuplicada|EdadEnMeses|InformacionPersonal|
+--------------+----+-----------+----+-------------+-----------+-------------------+
|         Alice|  25|   New York|EEUU|           50|        300|    Alice, New York|
|           Bob|  30|Los Angeles|EEUU|           60|        360|   Bob, Los Angeles|
|       Charlie|  22|    Chicago|EEUU|           44|        264|   Charlie, Chicago|
+--------------+----+-----------+----+-------------+-----------+-------------------+

